In [9]:
!. run_sixte.sh
from xspec import *
import os

Initialize Heasoft
Done


model  TBabs*bbodyrad
        value       delta           min     bot     top         max
          0.004      0.001          0          0     100000      1e+06
           0.06       0.01     0.0001      0.001        100        200
              1       0.01          0          0      1e+20      1e+24

In [ ]:
class myModel:
    def __init__(self,modelstring):
        pass

In [26]:
def create_model(modelstring:str, setPars:dict,frozen=None, save=None):
    """
    Example:
    ```python
    m = create_model("TBabs*(bbodyrad)", 
                 setPars={1: 4e-3,
                          2: 60e-3},
                          save=True)
    ```                        
    Args:
        modelstring (str): `xspec` model name 
        setPars (dict): set the parameters in dict format
        save (_type_, optional): Name of the file name of the model. If True, saves the modelstring as file name. Defaults to None.

    Returns:
        m: Model
    """
    Xset.chatter = 0
    Xset.abund = 'wilm'
    m = Model(modelstring, setPars=setPars)
    if (type(save)==str) and (save!=modelstring):
        if os.path.exists(f'{save}.xcm'):
            print('WARNING: overwriting model')
            os.remove(f'{save}.xcm')
        Xset.save(fileName=save, info='m')
    elif save:
        if os.path.exists(f'{modelstring}.xcm'):
            print('WARNING: overwriting model')
            os.remove(f'{modelstring}.xcm')
        Xset.save(fileName=modelstring, info='m')
    Xset.chatter = 10 
    return m

m = create_model("TBabs*(bbodyrad)", 
                 setPars={1:4e-3,2:60e-3},
                          save='mymodel')
m.show()


Model TBabs<1>*bbodyrad<2> Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   TBabs      nH         10^22    4.00000E-03  +/-  0.0          
   2    2   bbodyrad   kT         keV      6.00000E-02  +/-  0.0          
   3    2   bbodyrad   norm                1.00000      +/-  0.0          
________________________________________________________________________



In [43]:
!echo $SIXTE

/home/suro/SIXTE/installation


In [4]:
from astropy import units as u
import subprocess
class SIMPUT: 
    def __init__(self, XSPECFile, Simput, 
                 sixte_path='/home/suro/SIXTE/installation',
                 Src_Name=None, 
                 ra=0, 
                 dec=0, 
                 srcFlux=2e-11,
                 Elow=0.1,
                 Eup=10,
                 Nbins=1000,
                 logEgrid="yes",
                 Emin=0.2,
                 Emax=2):
        # required
        self.XSPECFile = XSPECFile # input model
        self.Simput = Simput #output SIMPUT file
        
        # optional default
        self.sixte_path = sixte_path
        self.Src_Name = Src_Name
        self.ra = ra
        self.dec = dec
        self.srcFlux = srcFlux
        self.Elow = Elow
        self.Eup = Eup
        self.Nbins = Nbins
        self.logEgrid = logEgrid
        self.Emin = Emin
        self.Emax = Emax
    
    def generate(self):
        simputfile_cmd = [
            f"{self.sixte_path}/bin/simputfile",
            f"Simput={self.Simput}",
            f"Src_Name={self.Src_Name}",
            f"RA={self.ra}",
            f"Dec={self.dec}",
            f"srcFlux={self.srcFlux}",
            f"Elow={self.Elow}",
            f"Eup={self.Eup}",
            f"NBins={self.Nbins}",
            f"logEgrid={self.logEgrid}",
            f"Emin={self.Emin}",
            f"Emax={self.Emax}",
            f"XSPECFile={self.XSPECFile}",
        ]
        try:
            subprocess.run(simputfile_cmd, check=True, executable="/bin/bash")
            print(f"SIMPUT file '{self.Simput}' successfully created.")
        except subprocess.CalledProcessError as e:
            print(f"Error creating SIMPUT file: {e}")
        except FileNotFoundError:
            print(f"Error: `simputfile` command not found in {self.sixte_path}.")

    
s = SIMPUT(XSPECFile="mymodel.xcm", Simput="file.fits", Src_Name='rxj1856')
s.generate()

Error creating SIMPUT file: Command '['/home/suro/SIXTE/installation/bin/simputfile', 'Simput=file.fits', 'Src_Name=rxj1856', 'RA=0', 'Dec=0', 'srcFlux=2e-11', 'Elow=0.1', 'Eup=10', 'NBins=1000', 'logEgrid=yes', 'Emin=0.2', 'Emax=2', 'XSPECFile=mymodel.xcm']' returned non-zero exit status 127.


/home/suro/SIXTE/installation/bin/simputfile: Simput=file.fits: No such file or directory


In [20]:
class SIXTEsim:
    def __init__(self, name):
        self.name = name
        
    def generateFolder(self):
        if not os.path.exists(self.name):
            os.mkdir(self.name)
            os.mkdir(os.path.join(self.name, 'model'))
            os.mkdir(os.path.join(self.name, 'simput'))
            
    def create_model(self, modelstring:str, setPars:dict=None, frozen=None, save=None):
        Xset.chatter = 0
        Xset.abund = 'wilm'
        m = Model(modelstring, setPars=setPars)
        if (type(save)==str) and (save!=modelstring):
            if os.path.exists(f'{save}.xcm'):
                print('WARNING: overwriting model')
                os.remove(f'{save}.xcm')
            Xset.save(fileName=save, info='m')
        elif save:
            if os.path.exists(f'{modelstring}.xcm'):
                print('WARNING: overwriting model')
                os.remove(f'{modelstring}.xcm')
            Xset.save(fileName=modelstring, info='m')
        Xset.chatter = 10 
        return m
    
    def create_SIMPUT(self):
        s = SIMPUT(XSPECFile=f"{self.name}/model/mymodel.xcm", Simput=f"{self.name}/simput/file.fits", Src_Name='rxj1856')
        s.generate()
        

simulation = SIXTEsim('firstsimulation')
simulation.generateFolder()
simulation.create_model(modelstring="TBabs*bbodyrad", save=f"{simulation.name}/model/mymodel.xcm")
simulation.create_SIMPUT()



Error creating SIMPUT file: Command '['/home/suro/SIXTE/installation/bin/simputfile', 'Simput=firstsimulation/simput/file.fits', 'Src_Name=rxj1856', 'RA=0', 'Dec=0', 'srcFlux=2e-11', 'Elow=0.1', 'Eup=10', 'NBins=1000', 'logEgrid=yes', 'Emin=0.2', 'Emax=2', 'XSPECFile=firstsimulation/model/mymodel.xcm']' returned non-zero exit status 127.


/home/suro/SIXTE/installation/bin/simputfile: Simput=firstsimulation/simput/file.fits: No such file or directory
